###  함수 설명

- `KakaoArena.Get_df` : 플레이리스트 id / 컬럼 전처리 후 데이터 프레임
- `KakaoArena.Merged_table` : 클러스터링 전 , 테이블 합치기
- `KakaoArena.Clustering_km` : word2vec 후 kmeans 로 클러스터링
- `KakaoArena.Get_cluster_matrix` : 메트릭스를 만들기
- `KakaoArena.Select_candidate` : 후보 플레이리스트 만들기
- `KakaoArena.Get_candidate` : 추천할 노래와 태그를 가져오는 함수
- `KakaoArena.Filter_date` : 날짜를 필터해주는 함수

## 불러오기

In [3]:
import pandas as pd 
import KakaoArena.Filter_date as fdate
import KakaoArena.Merged_table as mt # 후보군 테이블 merge 
import KakaoArena.Clustering_km as km
import KakaoArena.Get_cluster_matrix as gcm
import KakaoArena.Select_candidate as sc
import os

In [5]:
# 데이터 불러오기

genre = pd.read_json('data/genre_gn_all.json', typ = 'series', encoding = "utf-8")
meta = pd.read_json('data/song_meta.json', typ = 'frame', encoding="utf-8")
train = pd.read_json('data/train.json', typ = 'frame',encoding="utf-8")
val = pd.read_json('data/val.json', typ = 'frame',encoding="utf-8")
test = pd.read_json('data/test.json', typ = 'frame',encoding="utf-8")

complex_ = pd.concat([ train , val] , axis = 0 )
complex_ = pd.concat([ complex_ , test ] , axis = 0 )

In [6]:
# meta data  date 필터링 해주기 
filter_date = fdate.Filter_date( complex_ , meta )
meta = filter_date.kill_zeros()

# meta.to_pickle("data/done_filter_meta.pickle")
# meta = pd.read_pickle( 'data/done_filter_meta.pickle') # 한번했으면 이것만 불러오기

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
merged_t = mt.Merged_table( complex_, genre, meta )
clu_km = km.Clustering( complex_, meta )

tag_title_df = merged_t.tag_title( )

tag_title_merge


## 클러스터링 하는 함수

In [9]:
merged_t = mt.Merged_table( complex_, genre, meta )
clu_km = km.Clustering( complex_, meta )

# 태그와 타이틀 로 이루어진 데이터 프레임 만드는 함수
tag_title_df = merged_t.tag_title()
tag_title_df.to_pickle("data/tag_title_df.pickle")


# A. Song  : complex_
song_clu = clu_km.clustering_song( clu = 200 )
song_clu.to_pickle("data/clu_song_emb_200.pickle")

# B. word 
tag_gnr_title_clu = clu_km.clustering_( merged_t.tag_gnr_title() , clu = 100)
tag_gnr_title_clu.to_pickle("data/clu_tag_gnr_title_emb_100.pickle")

# C. singer 
singer_clu = clu_km.clustering_( merged_t.title_singer() , clu = 100)
singer_clu.to_pickle("data/clu_singer_emb_100.pickle")


# D. tag
tag_clu = clu_km.clustering_tag( clu = 30  )
tag_clu.to_pickle("data/clu_tag_emb_30.pickle")


# E. dtl_genre 원핫 인코딩을 했으므로 이 단계 생략


# F . album  아래에서 df 랑 clustering이랑 코드가 합쳐져있음. 
emb_df , album_df  = clu_km.clustering_album( clu = 100 )
emb_df.to_pickle('data/clu_album_emb100.pickle' )
album_df.to_pickle('data/album_df_.pickle') # 데이타 프레임 윗단계

tag_title_merge


KeyboardInterrupt: 

## matrix와 후보 플레이리스트 선별하기

In [10]:
matrix = gcm.Get_cluster_matrix( complex_, meta, genre )
candidate = sc.Select_candidate()

**A. song matrix & candidate**

In [13]:
song_vec = pd.read_pickle( 'data/clu_song_emb_200.pickle') # train + val + test 기반 임베딩
song_vec = song_vec[['song_id' , 'label'] + list(song_vec.columns[3:12])]

train_song = matrix.get_cluster_matrix( train , song_vec , by = "song")
val_song = matrix.get_cluster_matrix( val , song_vec , by = "song")

# # 최종!
# # test_song = gcm.get_cluster_matrix( test , song_vec  , by = 'song' ) 

sim_ply = candidate.dist_get_sim_ply( 0 ,  3000 , val_ply = val_song , train_ply = train_song , tn = 30 )
candidate.save_to_pickle( sim_ply , "candi/res_ver35_song.pickle")

3000it [00:13, 229.81it/s]


3000 만큼 했다


3000it [00:13, 226.51it/s]


6000 만큼 했다


3000it [00:13, 224.80it/s]


...

3000it [00:13, 227.67it/s]


21000 만큼 했다


2015it [00:08, 226.27it/s]


24000 만큼 했다


FileNotFoundError: [Errno 2] No such file or directory: 'candi/res_ver35_song.pickle'

In [ ]:
sim_ply

**B. word(gnr+tag+title) matrix & candidate**

In [ ]:

#임베딩된 개별 단어 df
word_vec = pd.read_pickle( 'data/clu_tag_gnr_title_emb_100.pickle' )

total_word = matrix.get_cluster_matrix( merged_t.tag_gnr_title() , word_vec , by = 'word')

val_word = total_word.loc[ total_word.id.isin( list(val.id.values) ) ] # word ( val )
train_word = total_word.loc[ total_word.id.isin( list(train.id.values ) ) ] # word ( 

# 최종
#test_word = total_word.loc[ total_word.id.isin( list(test.id.values ) ) ]
# del total_word #( 메모리 최적화를 위해 )

sim_ply = candidate.dist_get_sim_ply( 0 ,  3000 , val_ply = val_word , train_ply = train_word , tn = 30 )
candidate.save_to_pickle( sim_ply , 'candi/res_ver40_word.pickle')

**C. singer matrix & candidate**

In [ ]:
#임베딩된 개별 단어,가수
singer_vec = pd.read_pickle( 'data/clu_singer_emb_100.pickle')

total_singer = matrix.get_cluster_matrix( merged_t.title_singer() , singer_vec , by = 'word')
val_singer = total_singer.loc[ total_singer.id.isin( list(val.id.values) ) ] # singer 
train_singer = total_singer.loc[ total_singer.id.isin( list(train.id.values) ) ] # sin

#최종!
# test_singer = total_singer.loc[ total_singer.id.isin( list(test.id.values) ) ] 
# del total_word # 메모리 최적화


sim_ply = candidate.dist_get_sim_ply( 0 ,  3000 , val_ply = val_singer , train_ply = train_singer , tn = 30 )
candidate.save_to_pickle( sim_ply , 'candi/res_ver40_singer.pickle')

**D. tag matrix & candidate**

In [ ]:
# 태그로만 이루어진 ply df
tag_df = pd.read_pickle('tag_df.pickle') # clustring_tag 함수를 실행시키면 자동으로 저장됨

# 임베딩된 태그들
tag_vec = pd.read_pickle('data/clu_tag_emb_30.pickle') # train , val , test 모두 합쳐서 embeding

total_tag = matrix.get_cluster_matrix( tag_df , tag_vec , by = 'tag')
val_tag = total_tag.loc[ total_tag.id.isin( list(val.id.values) )]
train_tag = total_tag.loc[ total_tag.id.isin( list(train.id.values) )]

#최종
# test_tag = total_tag.loc[ total_tag.id.isin( list(test.id.values) )]

sim_ply = candidate.dist_get_sim_ply( 0 ,  3000 , val_ply = val_tag , train_ply = train_tag , tn = 30 )
candidate.save_to_pickle( sim_ply , 'candi/res_ver40_tag.pickle')

**E. genre matrix & candidate**

In [ ]:
total_gnr = matrix.get_genre_matrix( complex_ ,meta, genre )
val_gnr = total_gnr.loc[ total_gnr.id.isin( list(val.id.values) ) ] 
train_gnr = total_gnr.loc[ total_gnr.id.isin( list(train.id.values) ) ] 

#최종!
# train_gnr = total_gnr.loc[ total_gnr.id.isin( list(train.id.values) ) ] # 
# del train_gnr # 메모리 최적화

sim_ply = candidate.dist_get_sim_ply( 0 ,  3000 , val_ply = val_gnr , train_ply = train_gnr , tn = 30 )
candidate.save_to_pickle( sim_ply , 'candi/res_ver40_gnr.pickle')

**F. album matrix & candidate**

In [ ]:
album_df = pd.read_pickle('data/album_df.pickle')
album_vec = pd.read_pickle('data/clu_album_emb100.pickle')


total_album = matrix.get_cluster_matrix( album_df , album_vec , by = 'album')
train_album = total_album.loc[ total_album.id.isin( train.id.values ) ]
val_album = total_album.loc[ total_album.id.isin( val.id.values ) ]

# 최종 
# test_album = album_df.loc[ album_df.id.isin( train.id.values ) ]

sim_ply = candidate.dist_get_sim_ply( 0 , 3000 , val_ply = val_album , train_ply = train_album ,tn = 30 )
candidate.save_to_pickle( sim_ply , 'candi/res_ver40_album.pickle')

**G. Ply matrix**

In [ ]:
# ply matrix 

train_ply = pd.merge( train_song , train_word , on = 'id' , how = 'inner')
train_ply = pd.merge( train_ply , train_singer , on = 'id' , how = 'inner')
train_ply = pd.merge( train_ply , train_gnr , on = 'id' , how = 'inner')
train_ply = pd.merge( train_ply , train_tag , on = 'id' , how = 'inner')
train_ply = pd.merge( train_ply , train_album , on = 'id' , how = 'inner')

val_ply = pd.merge( val_song , val_word , on = 'id' , how = 'inner')
val_ply = pd.merge( val_ply , val_singer , on = 'id' , how = 'inner')
val_ply = pd.merge( val_ply , val_gnr , on = 'id' , how = 'inner')
val_ply = pd.merge( val_ply , val_tag , on = 'id' , how = 'inner')
val_ply = pd.merge( val_ply , val_album , on = 'id' , how = 'inner')

sim_ply = candidate.dist_get_sim_ply( 0 ,  3000 , val_ply = val_ply , train_ply = train_ply , tn = 30 )
candidate.save_to_pickle( sim_ply , 'candi/res_ver40_ply.pickle')

## 후보 플레이리스트에서 후보 노래와 태그를 뽑기

In [ ]:
import KakaoArena.Get_candidate as gc
get_songtag = gc.Get_candidate(train, tag_title_df)

In [ ]:
# 위에서 저장한 pickle 을 불러오기

s_sim = candidate.load_pickle( 'candi/res_ver35_song.pickle' )
w_sim = candidate.load_pickle( 'candi/res_ver35_word.pickle' )
sin_sim = candidate.load_pickle( 'candi/res_ver35_singer.pickle' )
t_sim = candidate.load_pickle( 'candi/res_ver35_tag.pickle' )
g_sim = candidate.load_pickle( 'candi/res_ver35_gnr.pickle' )

ply_sim = candidate.load_pickle( 'candi/res_ver35_ply.pickle' )

**후보곡과 태그를 뽑은 후, 날짜를 필터링해 줌**

In [ ]:
s_can = get_songtag.get_candidate( s_sim , val, by ="set" )
s_can = fdate.filter_date( s_can )

w_can = get_songtag.get_candidate( w_sim , val, by ="set" )
w_can = fdate.filter_date( w_can  )

sin_can = get_songtag.get_candidate( sin_sim , val, by ="set" )
sin_can = fdate.filter_date( sin_can)

t_can = get_songtag.get_candidate( t_sim , val, by ="set")
t_can = fdate.filter_date( t_can )

g_can = get_songtag.get_candidate( g_sim , val, by ="set")
g_can = fdate.filter_date( g_can )

ply_can = get_songtag.get_candidate( ply_sim , val, by ="set" )
ply_can = fdate.filter_date( ply_can)